selenium으로 네이버 블로그 크롤링하기

In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
import requests
from bs4 import BeautifulSoup

In [6]:
pip install --upgrade selenium

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\nt550\anaconda3\envs\textmining\lib\site-packages\py_hanspell-1.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [2]:
path = r"C:\Users\NT550\Desktop\chromedriver.exe"
s = Service(path)

### 셀레니움으로 네이버 블로그 url가져오는 코드

In [30]:
def selenium_search(search_keyword):
    # Selenium을 이용해 네이버 홈페이지 열기
    driver = webdriver.Chrome(service = s)  # 크롬 드라이버 경로 설정
    driver.get('https://www.naver.com')

    # 네이버 검색창에 "'솥밥밀키트'" 입력 후 검색 실행
    search_box = driver.find_element(By.CSS_SELECTOR, 'input#query')  # 검색창 요소 찾기 (CSS 선택자 사용)
    search_box.send_keys(f'"{search_keyword}"')  # 검색어 입력
    search_box.send_keys(Keys.RETURN)  # Enter 키 누르기
    # 링크 클릭
    # 링크 클릭 (VIEW)
    link = driver.find_element(By.PARTIAL_LINK_TEXT, 'VIEW')  # 링크 텍스트 일부로 요소 찾기
    link.click()

    # 링크 클릭 (블로그)
    link2 = driver.find_element(By.PARTIAL_LINK_TEXT, '블로그')  # 링크 텍스트 일부로 요소 찾기
    link2.click()
    
    # 스크롤 내리기
    SCROLL_PAUSE_TIME = 1  # 스크롤 간격 (초 단위)
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    blog_links = driver.find_elements(By.CSS_SELECTOR, 'a.api_txt_lines.total_tit')  # 블로그 주소 요소 찾기 (CSS 선택자 사용)
    # 블로그 주소 출력
    blog_urls = []
    for link in blog_links:
        blog_urls.append(link.get_attribute('href'))    
    driver.quit()
    return blog_urls



In [41]:
blog_urls = selenium_search("솥밥 밀키트")

### 가져온 url로 iframe안에 숨겨진 url 가져오기

In [27]:
def selenium_real_blog_url(blog_urls):
    driver = webdriver.Chrome(service = s)  # 크롬 드라이버 경로 설정
    real_blog_urls = []
    for url in blog_urls:
        driver.get(f"{url}")
        real_blog_link = driver.find_element(By.CSS_SELECTOR,"iframe")
        real_blog_url = real_blog_link.get_attribute('src')
        if real_blog_url.startswith("https://blog.naver.com/PostView.naver"):
            real_blog_urls.append(real_blog_url)
        else:
            pass
        
    driver.quit()    
    return real_blog_urls

In [28]:
real_blog_urls = selenium_real_blog_url(blog_urls)

KeyboardInterrupt: 

### 블로그 내용 크롤링에 필요한 logno번호 분리

In [12]:
def log_num(real_blog_urls):
    log_num = []
    for url in real_blog_urls:
        match = re.search(r"logNo=(\d+)", url)
        if match:
            log_no = match.group(1)
            log_num.append(log_no)
    return log_num

In [13]:
log_num = log_num(real_blog_urls)

In [14]:
print(len(blog_urls),len(real_blog_urls),len(log_num))

193 192 192


In [15]:
real_blog_urls

['https://blog.naver.com/PostView.naver?blogId=wlsdud6818&logNo=223138373216&redirect=Dlog&widgetTypeCall=true&directAccess=false',
 'https://blog.naver.com/PostView.naver?blogId=whtjsdud1616&logNo=222880250515&redirect=Dlog&widgetTypeCall=true&directAccess=false',
 'https://blog.naver.com/PostView.naver?blogId=nuri_yuri&logNo=222863249402&redirect=Dlog&widgetTypeCall=true&directAccess=false',
 'https://blog.naver.com/PostView.naver?blogId=miwoomiwoo27&logNo=222882657940&redirect=Dlog&widgetTypeCall=true&directAccess=false',
 'https://blog.naver.com/PostView.naver?blogId=lala0112&logNo=222890937235&redirect=Dlog&widgetTypeCall=true&directAccess=false',
 'https://blog.naver.com/PostView.naver?blogId=twinj2767&logNo=222737405328&redirect=Dlog&widgetTypeCall=true&directAccess=false',
 'https://blog.naver.com/PostView.naver?blogId=minji8635&logNo=222942386982&redirect=Dlog&widgetTypeCall=true&directAccess=false',
 'https://blog.naver.com/PostView.naver?blogId=mynamco&logNo=222833215348&red

### 리얼 주소랑 로그번호로 각 블로그의 본문내용 크롤링

In [16]:
def selenium_txt_crawling(real_blog_urls,log_num):
    path = r"C:\Users\NT550\Desktop\chromedriver.exe"
    driver = webdriver.Chrome(service = s)  # 크롬 드라이버 경로 설정
    blog_text = {}
    cnt = 0
    # blog_text = {'url1':[], 'url2':[]}
    # blog_text['url1'].append(value)
    for num, url in enumerate(real_blog_urls):
        driver.get(f"{url}")
        cnt += 1
        tmp = [] 
        try:
            temp = driver.find_elements(By.CSS_SELECTOR, "p")
            temp = [i.text for i in temp]
            blog_text[url] = [",".join(temp)]
        except:
                try:
                    text = driver.find_element(By.XPATH, f'//*[@id="post-view{log_num[num]}"]/div/div/div[2]').text
                    tmp.append(text)
                    text = driver.find_element(By.XPATH, f'//*[@id="post-view{log_num[num]}"]/div/div[2]').text
                    tmp.append(text)
                    blog_text[url] = tmp
                except:
                    print("블로그 내용을 가져올 수 없음:", url)
                    try:
                        text = driver.find_element(By.XPATH, f'//*[@id="post-view{log_num[num]}"]/div/div/div[2]').text
                        tmp.append(text)
                        blog_text[url] = tmp
                    except:
                        try:
                            print("마지막 수단으로 블로그 내용을 가져옴:", cnt, url)
                            text = driver.find_element(By.XPATH, f'//*[@id="post-view{log_num[num]}"]/div/div[2]').text
                            tmp.append(text)
                            blog_text[url] = tmp
                        except:
                            print("제거바람",cnt)
                            pass
    return blog_text

In [17]:
blog_text = selenium_txt_crawling(real_blog_urls,log_num)

블로그 내용을 가져올 수 없음: https://blog.naver.com/PostView.naver?blogId=miwoomiwoo27&logNo=222882657940&redirect=Dlog&widgetTypeCall=true&directAccess=false
블로그 내용을 가져올 수 없음: https://blog.naver.com/PostView.naver?blogId=lala0112&logNo=222890937235&redirect=Dlog&widgetTypeCall=true&directAccess=false
블로그 내용을 가져올 수 없음: https://blog.naver.com/PostView.naver?blogId=en_ji&logNo=222879360270&redirect=Dlog&widgetTypeCall=true&directAccess=false
블로그 내용을 가져올 수 없음: https://blog.naver.com/PostView.naver?blogId=sya76&logNo=222238126762&redirect=Dlog&widgetTypeCall=true&directAccess=false
블로그 내용을 가져올 수 없음: https://blog.naver.com/PostView.naver?blogId=heroheropsw&logNo=222239997553&redirect=Dlog&widgetTypeCall=true&directAccess=false
블로그 내용을 가져올 수 없음: https://blog.naver.com/PostView.naver?blogId=tjswjd1981&logNo=223057460972&redirect=Dlog&widgetTypeCall=true&directAccess=false
블로그 내용을 가져올 수 없음: https://blog.naver.com/PostView.naver?blogId=niedlichej&logNo=222436494625&redirect=Dlog&widgetTypeCall=true&direct

In [18]:
blog_text

{'https://blog.naver.com/PostView.naver?blogId=wlsdud6818&logNo=223138373216&redirect=Dlog&widgetTypeCall=true&directAccess=false': [',,,,,,,내돈내산 마켓컬리 밀키트 추천 - 수린 한우사골솥밥 밀키트,안녕하세요 싱입니다.,오늘은 마켓컬리에서 솥밥밀키트가 있어서 한번 도전해보고싶어서 사봤어요 >< 저의 첫 냄비밥이에요 ㅎㅎ,출처 : 마켓컬리 앱,,저는 18,900원 주고 샀는데 ㅎㅎ 글쓸때보니 ㅋㅋㅋ 10프로 세일하네요 ㅎㅎ 수린의 한우 사골 솥밥 밀키트 2인분은 1++ 한우 설깃살이 들어간답니다 ><,,출처 : 마켓컬리 앱,,마켓컬리앱에서 조리법이 두가지가 있더라구요. 저는 냄비밥으로 시도해봤어요 ㅎㅎ,,요건 냉장보관이여서 배송온 후에는 먹기전까지 냉장보관을 해줬어요.,,,원재료명이 적혀있어서 사진을 찍어봤답니다.,,위에 앱에도 적혀있었던 조리법이 포장지에도 있어서 ㅎㅎ 보고 따라만 하면 된답니다.,,,안에는 재료들이 들어있었어요.,,,1++ 한우 설깃살, 사골육수, 국산 쌀, 표고버섯, 쪽파가 들어있었어요.,,재료들이 하나씩 진공포장 되어있어서 좋았어요.,,,먼저 쌀을 씻어줄께요 ><,,쌀을 씻은 후에 한우 사골을 넣고 표고버섯도 넣어줬고 20분정도 쌀을 불려줬어요.,,,저는 집에 ㅎㅎ 솥이 있어서 이곳에 솥밥을 해봤어요 ㅎㅎ,,,불은 약불로 20분정도 끓여주고 5분정도 뜸을 들여주면 된답니다.,,밥을 하는동안 쪽파를 잘게 썰어줬어요 >< 가위로 자르니 ㅎㅎ 크기가 일정하더라구요 ㅎㅎ,,,한우 설깃살은 너무 오래 볶으면 질겨진다고해서 핏기가 없어질때쯤 까지만 볶아줬어요.,,,준비는 이제 끝이에요 ㅎㅎ 이제 밥만 되면 된답니다 ㅎㅎ,,,저는 여기에 ㅋㅋㅋ 소고기 등심을 같이 구워서 먹을려구요 ㅋㅋㅋ,,,고기도 잘 구워졌네요 ㅎㅎ,,약불로 20분 끓인후 5분 뜸을 들이고 뚜껑을 열어봤어요 ><,,,밥이 고슬고슬 잘됐답니다 >< 냄비밥 처음인데 ㅋㅋㅋ 안

### 데이터 프레임으로 변환

In [19]:
naver_blog_text = pd.DataFrame(list(blog_text.items()), columns=["주소", "내용"])

### 본문 내용 전처리(한글,영어,숫자,.)제외하고 제거

In [20]:
# 먼저 리스트의 원소들을 문자열로 결합하여 하나의 큰 문자열로 만들어줍니다.
naver_blog_text["내용"] = naver_blog_text["내용"].apply(lambda x: ''.join(x))
naver_blog_text["변환된 내용"] = naver_blog_text["내용"].apply(lambda x: re.sub('\\n', '', x))
naver_blog_text["변환된 내용"] = naver_blog_text["변환된 내용"].apply(lambda x: re.sub(r'[^\w\sㄱ-ㅎㅏ-ㅣ가-힣.]', '', x))

In [21]:
naver_blog_text

주소  \
0    https://blog.naver.com/PostView.naver?blogId=w...   
1    https://blog.naver.com/PostView.naver?blogId=w...   
2    https://blog.naver.com/PostView.naver?blogId=n...   
3    https://blog.naver.com/PostView.naver?blogId=m...   
4    https://blog.naver.com/PostView.naver?blogId=l...   
..                                                 ...   
187  https://blog.naver.com/PostView.naver?blogId=j...   
188  https://blog.naver.com/PostView.naver?blogId=d...   
189  https://blog.naver.com/PostView.naver?blogId=m...   
190  https://blog.naver.com/PostView.naver?blogId=c...   
191  https://blog.naver.com/PostView.naver?blogId=t...   

                                                    내용  \
0    ,,,,,,,내돈내산 마켓컬리 밀키트 추천 - 수린 한우사골솥밥 밀키트,안녕하세요 ...   
1    육아하는 RN 널봄맘\n요리똥손인 제가 환절기를 앞두고 식구들 몸보신을 위해\n뭘 ...   
2    ,,,,,,,[혼족의 제왕] 자취끼니 서포터즈 끼니키트 구성은 어떨까? 1인가구요리...   
3    전복솥밥 키트가 있더라구요. 몸 보신하고 싶어서 최상급 전복 사서 직접 만들어 먹으...   
4    연년생 형제들을 키우느라 정신 없는 요즘,\n진짜 세명의 인생을 살아가고 있는 것 ...   
..                                                 ...   
187  날씨도 쌀쌀해진 요즘 뭐 드시나요?\n집에서 간단히 해먹을만한거 추천좀 해달라고\n...   
188  짝지가 출장으로 부재중이다보니\n집에서 집밥다운 집밥을 먹기가 힘들어지는 것 같아요...   
189  ,,,,,,,맛과 멋을 찾아 사람들에게 즐거움을 \n주는 블로그입니다. \n유용한 ...   
190  ,,,,,,,21-07-2021(+2380)7살 아이 식단, 7살 어린이 하루식단,...   
191  영양만점 간편식 식탁위로 맛있어요\nhttp://식탁위로.com\n#캠핑요리 #식탁...   

                                                변환된 내용  
0    내돈내산 마켓컬리 밀키트 추천  수린 한우사골솥밥 밀키트안녕하세요 싱입니다.오늘은 ...  
1    육아하는 RN 널봄맘요리똥손인 제가 환절기를 앞두고 식구들 몸보신을 위해뭘 조리해볼...  
2    혼족의 제왕 자취끼니 서포터즈 끼니키트 구성은 어떨까 1인가구요리키트 솥밥 밀키트 ...  
3    전복솥밥 키트가 있더라구요. 몸 보신하고 싶어서 최상급 전복 사서 직접 만들어 먹으...  
4    연년생 형제들을 키우느라 정신 없는 요즘진짜 세명의 인생을 살아가고 있는 것 같아밥...  
..                                                 ...  
187  날씨도 쌀쌀해진 요즘 뭐 드시나요집에서 간단히 해먹을만한거 추천좀 해달라고지인들과 ...  
188  짝지가 출장으로 부재중이다보니집에서 집밥다운 집밥을 먹기가 힘들어지는 것 같아요ㅠ_...  
189  맛과 멋을 찾아 사람들에게 즐거움을 주는 블로그입니다. 유용한 정보도 많이 있으니 ...  
190  2107202123807살 아이 식단 7살 어린이 하루식단 21072021 7살 어...  
191  영양만점 간편식 식탁위로 맛있어요http식탁위로.com캠핑요리 식탁위로간편식 소주안...  

[192 rows x 3 columns]

### 간단한 전처리가 끝난 파일 csv파일로 꺼내기

In [22]:
naver_blog_text.to_csv(r"naver_blog_text_scaling.csv",encoding= "utf-8-sig", index= False)